<a href="https://colab.research.google.com/github/corneliarsm/Classification_Using_Tensorflow/blob/master/belajarGIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import TensorFlow and other libraries**

In [0]:
pip install -q sklearn

In [0]:
import numpy as np
import pandas as pd
from __future__ import absolute_import, division, print_function, unicode_literals

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers

**Use Pandas to create a dataframe**

In [0]:
url = 'https://raw.githubusercontent.com/putricinto/Machine-Learning-Project---MLCC/master/bank.csv'
bank = pd.read_csv(url)

In [0]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [0]:
bank['deposit'] = bank['deposit'].replace('yes', 1)
bank['deposit'] = bank['deposit'].replace('no', 0)
bank['deposit']

0        1
1        1
2        1
3        1
4        1
        ..
11157    0
11158    0
11159    0
11160    0
11161    0
Name: deposit, Length: 11162, dtype: int64

**Split the dataframe into train, validation, and test**

In [0]:
train, test = train_test_split(bank, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

9040 train examples
1005 validation examples
1117 test examples


**Create an input pipeline using tf.data**

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def bank_to_dataset(bank, shuffle=True, batch_size=32):
  bank = bank.copy()
  labels = bank.pop('deposit')
  ds = tf.data.Dataset.from_tensor_slices((dict(bank), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(bank))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 5
train_ds = bank_to_dataset(train, batch_size=batch_size)
val_ds = bank_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = bank_to_dataset(test, shuffle=False, batch_size=batch_size)

**Understand the input pipeline**

In [0]:
for feature_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))

RuntimeError: ignored

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution() #So that, this code can work -> marks = feature_column.numeric_column("marks")
from tensorflow import feature_column
from tensorflow.keras import layers

In [0]:
data = {'marks': [55,21,63,88,74,54,95,41,84,52],
        'grade': ['average','poor','average','good','good','average','good','average','good','average'],
        'point': ['c','f','c+','b+','b','c','a','d+','b+','c']}

In [90]:
df = pd.DataFrame(data)
df

,marks,grade,point
0,55,average,c
1,21,poor,f
2,63,average,c+
3,88,good,b+
4,74,good,b
5,54,average,c
6,95,good,a
7,41,average,d+
8,84,good,b+
9,52,average,c


In [92]:
marks = feature_column.numeric_column("marks")
demo(marks)

AttributeError: ignored

In [93]:
marks_buckets = feature_column.bucketized_column(marks, boundaries=[30,40,50,60,70,80,90])
demo(marks_buckets)

AttributeError: ignored

In [9]:
grade = feature_column.categorical_column_with_vocabulary_list(
      'grade', ['poor', 'average', 'good'])
grade_one_hot = feature_column.indicator_column(grade)
demo(grade_one_hot)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [10]:
point = feature_column.categorical_column_with_vocabulary_list(
 'point', df['point'].unique())
point_one_hot = feature_column.indicator_column(point)
demo(point_one_hot)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]
ERROR! Session/line number was not unique in database. History logging moved to new session 61


In [12]:
point_embedding = feature_column.embedding_column(point, dimension=4)
demo(point_embedding)

[[ 0.60089236  0.28356612  0.21711372 -0.441494  ]
 [-0.6634747   0.26396716  0.46738845 -0.34748778]
 [-0.17475052 -0.04799718 -0.00643221 -0.00314473]
 [-0.4148967   0.5837896   0.42685306  0.18952645]
 [-0.4025464  -0.18875651 -0.90303165  0.10808443]
 [ 0.60089236  0.28356612  0.21711372 -0.441494  ]
 [-0.54682404  0.75314367  0.3397444   0.349551  ]
 [ 0.2003746  -0.26015043 -0.7478044   0.33892724]
 [-0.4148967   0.5837896   0.42685306  0.18952645]
 [ 0.60089236  0.28356612  0.21711372 -0.441494  ]]


In [18]:
point_hashed = feature_column.categorical_column_with_hash_bucket(
      'point', hash_bucket_size=4)
demo(feature_column.indicator_column(point_hashed))

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [16]:
crossed_feature = feature_column.crossed_column([marks_buckets, grade], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [0]:
df_bank = pd.DataFrame(bank)
df_bank

In [0]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(data).numpy())

In [74]:
age = tf.feature_column.numeric_column(key='age', dtype=tf.float32)
demo(age)

ValueError: ignored

In [43]:
age

NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [0]:
bank.columns

In [68]:
default = tf.feature_column.categorical_column_with_vocabulary_list(key='default')
default_one_hot = feature_column.indicator_column(default)
demo(default_one_hot)

TypeError: ignored

In [0]:
tf.compat.v1.feature_column.linear_model(
    features, feature_columns, units=1, sparse_combiner='sum',
    weight_collections=None, trainable=True, cols_to_vars=None
)

In [0]:
age= tf.feature_column.numeric_column('age')
day= tf.feature_column.numeric_column('day')
duration= tf.feature_column.numeric_column('duration')
campaign= tf.feature_column.numeric_column('campaign')
feature_columns = [
    tf.feature_column.indicator_column(age),
    tf.feature_column.indicator_column(day),
    tf.feature_column.indicator_column(duration),
    tf.feature_column.indicator_column(campaign)
]
feature_columns

In [0]:
feature_columns

In [0]:
bank.feature_column

In [0]:
policy_state = tf.feature_column.categorical_column_with_vocabulary_list('policy_state', [    'AL', 'CA', 'MI'])

modern_classic_ind = tf.feature_column.categorical_column_with_vocabulary_list('modern_classic_ind', ['0', '1'])

h_plus_ind = tf.feature_column.categorical_column_with_vocabulary_list('h_plus_ind', [ '0', '1'])

retention_ind = tf.feature_column.categorical_column_with_vocabulary_list('retention_ind', [  '0', '1'])

feature_columns = [
    tf.feature_column.indicator_column(policy_state),
    tf.feature_column.indicator_column(modern_classic_ind),
    tf.feature_column.indicator_column(h_plus_ind)
]

**AAAAAAAAA GATAUUUUUU**

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.enable_eager_execution()

In [0]:
data = {'marks': [55,21,63,88,74,54,95,41,84,52],
        'grade': ['average','poor','average','good','good','average','good','average','good','average'],
        'point': ['c','f','c+','b+','b','c','a','d+','b+','c']}

In [26]:
url2 = 'https://raw.githubusercontent.com/putricinto/Machine-Learning-Project---MLCC/master/bank.csv'
bank2 = pd.read_csv(url2)
df2 = pd.DataFrame(bank2)
df2.age

0        59
1        56
2        41
3        55
4        54
         ..
11157    33
11158    39
11159    32
11160    43
11161    34
Name: age, Length: 11162, dtype: int64

In [27]:
df = pd.DataFrame(data)
df.marks

0    55
1    21
2    63
3    88
4    74
5    54
6    95
7    41
8    84
9    52
Name: marks, dtype: int64

In [0]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(bank2).numpy())

In [0]:
age = feature_column.numeric_column("age")

In [0]:
day= tf.feature_column.numeric_column('day')

In [0]:
duration= tf.feature_column.numeric_column('duration')

In [0]:
campaign= tf.feature_column.numeric_column('campaign')

In [0]:
feature_columns = [
    tf.feature_column.indicator_column(age),
    tf.feature_column.indicator_column(day),
    tf.feature_column.indicator_column(duration),
    tf.feature_column.indicator_column(campaign)
]

In [37]:
feature_columns

[IndicatorColumn(categorical_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)),
 IndicatorColumn(categorical_column=NumericColumn(key='day', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)),
 IndicatorColumn(categorical_column=NumericColumn(key='duration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)),
 IndicatorColumn(categorical_column=NumericColumn(key='campaign', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None))]

In [0]:
demo(feature_columns)

# New Section AYO SEMANGATTTT

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.enable_eager_execution()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
url2 = 'https://raw.githubusercontent.com/putricinto/Machine-Learning-Project---MLCC/master/bank.csv'
bank2 = pd.read_csv(url2)
df2 = pd.DataFrame(bank2)
df2

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [0]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(bank2).numpy())

In [0]:
job = feature_column.categorical_column_with_vocabulary_list(
      'job',['admin.','blue-collar','entrepreneur','housemaid','management',
             'retired','self-employed','services','student','technician',
             'unemployed','unknown']
)
marital = feature_column.categorical_column_with_vocabulary_list(
      'marital',['divorced','married','single']
)
education = feature_column.categorical_column_with_vocabulary_list(
      'education',['primary','secondary','tertiary','unknown']
)
default = feature_column.categorical_column_with_vocabulary_list(
      'default',['no','yes']
)
housing = feature_column.categorical_column_with_vocabulary_list(
      'housing',['no','yes']
)
loan = feature_column.categorical_column_with_vocabulary_list(
      'loan',['no','yes']
)
contact = feature_column.categorical_column_with_vocabulary_list(
      'contact',['cellular','telephone','unknown']
)
month = feature_column.categorical_column_with_vocabulary_list(
      'month',['jan','feb','mar','apr','may','jun','jul','aug','sep','oct',
               'nov','dec']
)
poutcome = feature_column.categorical_column_with_vocabulary_list(
      'poutcome',['failure','success','unknown','other']
)

job_hot = feature_column.indicator_column(job)
marital_hot = feature_column.indicator_column(marital)
education_hot = feature_column.indicator_column(education)
default_hot = feature_column.indicator_column(default)
housing_hot = feature_column.indicator_column(housing)
loan_hot = feature_column.indicator_column(loan)
contact_hot = feature_column.indicator_column(contact)
month_hot = feature_column.indicator_column(month)
poutcome_hot = feature_column.indicator_column(poutcome)

In [0]:
age = feature_column.numeric_column("age")
balance = feature_column.numeric_column("balance")
day = feature_column.numeric_column("day")
duration = feature_column.numeric_column("duration")
campaign = feature_column.numeric_column("campaign")
pdays = feature_column.numeric_column("pdays")
previous = feature_column.numeric_column("previous")

In [15]:
bank2['deposit'] = bank2['deposit'].replace('yes', 1)
bank2['deposit'] = bank2['deposit'].replace('no', 0)
bank2['deposit']

0        1
1        1
2        1
3        1
4        1
        ..
11157    0
11158    0
11159    0
11160    0
11161    0
Name: deposit, Length: 11162, dtype: int64

In [0]:
x_data = bank2.drop('deposit',axis=1)
y_labels = bank2['deposit']
X_train, X_test, y_train, y_test = train_test_split(
      x_data,y_labels,test_size=0.3,random_state=101)

In [0]:
feat_cols = [
             job_hot,marital_hot,education_hot,default_hot,housing_hot,
             loan_hot,contact_hot,month_hot,poutcome_hot,age,balance,
             day,duration,campaign,pdays,previous]

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                 y=y_train,
                                                 batch_size=10,
                                                 num_epochs=None,
                                                 shuffle=True)

In [36]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp90ih5nl_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f25cb0eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [37]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp90ih5nl_/model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 235.048
INFO:tensorflow:loss = 144.11478, step = 101 (0.430 sec)
INFO:tensorflow:global_step/sec: 298.067
INFO:tensorflow:loss = 62.83639, step = 201 (0.337 sec)
INFO:tensorflow:global_step/sec: 294.678
INFO:tensorflow:loss = 164.30246, step = 301 (0.335 sec)
INFO:tensorflow:global_step/sec: 296.821
INFO:tensorflow:loss = 3.1333153, step = 401 (0.339 sec)
INFO:tensorflow:global_step/sec: 311.576
INFO:tensorflow:loss = 32.786644, step = 501 (0.320 sec)
INFO:tensorflow:global_step/sec: 307.794
INFO:tensorflow:loss = 8.689968, step = 601 (0.325 sec)
INFO:tensorflow:global_step/sec: 315.85
INFO:tensorflow:loss

In [0]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [39]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp90ih5nl_/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [40]:
predictions[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logistic': array([0.99996924], dtype=float32),
 'logits': array([10.387995], dtype=float32),
 'probabilities': array([3.0799089e-05, 9.9996924e-01], dtype=float32)}

In [0]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [42]:
final_preds[:20]

[1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1761
           1       0.81      0.83      0.82      1588

    accuracy                           0.83      3349
   macro avg       0.82      0.83      0.82      3349
weighted avg       0.83      0.83      0.83      3349



In [0]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=len(X_test),shuffle=False)

In [45]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1761
           1       0.81      0.83      0.82      1588

    accuracy                           0.83      3349
   macro avg       0.82      0.83      0.82      3349
weighted avg       0.83      0.83      0.83      3349



In [46]:
results = model.evaluate(eval_input_func)
results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-25T12:43:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp90ih5nl_/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-25-12:43:49
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8250224, accuracy_baseline = 0.5258286, auc = 0.8968106, auc_precision_recall = 0.8766301, average_loss = 0.587304, global_step = 5000, label/mean = 0.4741714, loss = 1966.8811, precision = 0.8058608, prediction/mean = 0.4928987, recall = 0.8312343
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp90ih5nl_/model.ckpt-5000


{'accuracy': 0.8250224,
 'accuracy_baseline': 0.5258286,
 'auc': 0.8968106,
 'auc_precision_recall': 0.8766301,
 'average_loss': 0.587304,
 'global_step': 5000,
 'label/mean': 0.4741714,
 'loss': 1966.8811,
 'precision': 0.8058608,
 'prediction/mean': 0.4928987,
 'recall': 0.8312343}

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
from tensorflow import feature_column
from tensorflow.keras import layers
tf.enable_eager_execution()